In [1]:
library(tidyverse)
library(readr)
library(ggplot2)
library(dplyr)
library(lubridate)
library(knitr)

players <- read_csv("data/players.csv")
sessions <- read_csv("data/sessions.csv")

colnames(players)
colnames(sessions)

merged_data <- left_join(sessions, players, by = "player")

player_summary <- merged_data %>%
  group_by(player) %>%
  summarise(
    total_sessions = n(),
    total_playtime = sum(duration),
    avg_playtime = mean(duration),
    first_day = min(as_date(start)),
    week1_playtime = sum(duration[as_date(start) <= first_day + days(7)])
  )

threshold <- quantile(player_summary$total_playtime, 0.75)

player_summary <- player_summary %>%
  mutate(
    high_contributor = total_playtime >= threshold
  )

eda_data <- left_join(player_summary, players, by = "player")

ggplot(eda_data, aes(x = total_sessions, y = total_playtime, color = high_contributor)) +
  geom_point() +
  labs(
    title = "Sessions vs Total Playtime",
    x = "Total Sessions",
    y = "Total Playtime (minutes)",
    color = "High Contributor"
  ) +
  theme_minimal()

eda_summary <- eda_data %>%
  group_by(high_contributor) %>%
  summarise(
    avg_playtime = mean(total_playtime),
    avg_sessions = mean(total_sessions),
    total_players = n()
  )

knitr::kable(eda_summary, digits = 2)

eda_data <- eda_data %>%
  select(player, total_sessions, total_playtime, avg_playtime, first_day, week1_playtime, high_contributor) %>%
  arrange(desc(total_playtime))

eda_data

glimpse(eda_data)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


ERROR: Error: 'data/players.csv' does not exist in current working directory ('/home/jovyan/work').
